In [1]:
import numpy as np
from model_MLP import MLP
import torch

In [11]:
config = np.array([[ 0.,          0.0534427,   0.05344266],
                    [ 0.,         -0.10604769,  1.05260506],
                     [ 0.,          1.05260499, -0.10604768],])
print(config)

[[ 0.          0.0534427   0.05344266]
 [ 0.         -0.10604769  1.05260506]
 [ 0.          1.05260499 -0.10604768]]


In [15]:
config = np.array([[0,0,0],[0,0,1],[0,1,0]])

In [2]:
#load the model
model = MLP()
model.load_state_dict(torch.load(r'weight\pes_model_weights.pth'))


RuntimeError: Error(s) in loading state_dict for MLP:
	Missing key(s) in state_dict: "layers.0.weight", "layers.0.bias", "layers.2.weight", "layers.2.bias", "layers.4.weight", "layers.4.bias", "layers.6.weight", "layers.6.bias". 
	Unexpected key(s) in state_dict: "atom_subnets.0.layers.0.weight", "atom_subnets.0.layers.0.bias", "atom_subnets.0.layers.2.weight", "atom_subnets.0.layers.2.bias", "atom_subnets.0.layers.4.weight", "atom_subnets.0.layers.4.bias", "atom_subnets.1.layers.0.weight", "atom_subnets.1.layers.0.bias", "atom_subnets.1.layers.2.weight", "atom_subnets.1.layers.2.bias", "atom_subnets.1.layers.4.weight", "atom_subnets.1.layers.4.bias", "atom_subnets.2.layers.0.weight", "atom_subnets.2.layers.0.bias", "atom_subnets.2.layers.2.weight", "atom_subnets.2.layers.2.bias", "atom_subnets.2.layers.4.weight", "atom_subnets.2.layers.4.bias". 

In [3]:
#convert to tensor
config = torch.tensor(config, dtype=torch.float32)
config = config.reshape(1,9)
#predict
output = model(config)
print(output)



NameError: name 'model' is not defined

In [5]:
# predict force using gradient at the bottom layer
config.requires_grad = True
output = model(config)
output.backward()
print(config.grad)

tensor([[ 134.7454,  813.2773, -999.0754,  -37.5762, -171.8288,  645.9871,
           -8.6931, -688.5799,  374.3125]])


In [4]:
from datageneration import *

In [12]:
topology = create_water_topology()
simulation = setup_simulation(topology)
config = np.array([[ 0.,          0.0534427,   0.05344266],
 [ 0.,         -0.10604769,  1.05260506],
 [ 0.,          1.05260499, -0.10604768],])
potential_energy, force = calculate_potential_energy_and_forces(simulation, config)

In [8]:
print(force)

[[ 0.          1.65714926  1.66899399]
 [ 0.         -1.40472912 -0.26263409]
 [ 0.         -0.25242013 -1.40635987]] kJ/(nm mol)


In [13]:
potential_energy

Quantity(value=1.2486228115449194e-05, unit=kilojoule/mole)

In [33]:
import torch
import torchani
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import os

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ani_model = torchani.models.ANI1x(periodic_table_index=True).to(device).double()
# Load the weight
weight_path = r"weight\ani_model_weights.pth"
ani_model.load_state_dict(torch.load(weight_path))

C:/Users/Lenovo/anaconda3/envs/OpenMM/Lib/site-packages/torchani\resources/


<All keys matched successfully>

In [40]:
config = np.array([[[ 0.,          0.0534427,   0.05344266],
                    [ 0.,         -0.10604769,  1.05260506],
                     [ 0.,          1.05260499, -0.10604768]]])
specieset = torch.tensor([[8, 1, 1]], device=device)
config = torch.tensor(config, requires_grad=True,dtype=torch.double)

In [41]:
energy = ani_model((specieset, config)).energies

In [42]:
derivative = torch.autograd.grad(energy.sum(), config)[0]
force = -derivative

In [43]:
print(force)

tensor([[[-0.0000, -0.7204, -0.7204],
         [-0.0000, -0.0479,  0.7683],
         [-0.0000,  0.7683, -0.0479]]], dtype=torch.float64)
